In [1]:
import pandas as pd
import numpy as np
import gc
import optuna
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import warnings
import joblib
import json

warnings.filterwarnings("ignore")

In [2]:
def load_data_with_scaling_encoding_dtypes():
    # Define dtypes for train_transaction
    dtypes_train_transaction = {
        'TransactionID': 'int32',
        'isFraud': 'int8',
        'TransactionDT': 'int32',
        'TransactionAmt': 'float32',
        'ProductCD': 'object',
        'card1': 'int16',
        'card2': 'float32',
        'card3': 'float32',
        'card4': 'object',
        'card5': 'float32',
        'card6': 'object',
        'addr1': 'float32',
        'addr2': 'float32',
        'dist1': 'float32',
        'dist2': 'float32',
        'P_emaildomain': 'object',
        'R_emaildomain': 'object',
        # Add all C, D, M, V features
        **{f'C{i}': 'float32' for i in range(1, 15)},
        **{f'D{i}': 'float32' for i in range(1, 16)},
        **{f'M{i}': 'object' for i in range(1, 10)},
        **{f'V{i}': 'float32' for i in range(1, 340)}
    }

    # Define dtypes for train_identity
    dtypes_train_identity = {
        **{f'id_{i:02}': 'float32' for i in range(1, 12)},
        'id_12': 'object',
        'id_13': 'float32',
        'id_14': 'float32',
        'id_15': 'object',
        'id_16': 'object',
        **{f'id_{i:02}': 'float32' for i in range(17, 40)},
        **{f'id_{i:02}': 'object' for i in range(23, 39)},
        'DeviceType': 'object',
        'DeviceInfo': 'object'
    }

    # Define dtypes for test datasets
    dtypes_test_transaction = dtypes_train_transaction.copy()
    del dtypes_test_transaction['isFraud']  # Test data does not contain 'isFraud'

    dtypes_test_identity = dtypes_train_identity.copy()

    # Read in datasets with dtypes
    train_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_transaction.csv', 
        dtype=dtypes_train_transaction
    )
    train_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_identity.csv', 
        dtype=dtypes_train_identity
    )
    test_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_transaction.csv', 
        dtype=dtypes_test_transaction
    )
    test_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_identity.csv', 
        dtype=dtypes_test_identity
    )

    # Standardize column names
    test_identity.columns = test_identity.columns.str.replace('-', '_')
    test_transaction.columns = test_transaction.columns.str.replace('-', '_')

    # Merge datasets
    train = train_transaction.merge(train_identity, how='left', on='TransactionID')
    test = test_transaction.merge(test_identity, how='left', on='TransactionID')

    # Free up memory
    del train_transaction, train_identity, test_transaction, test_identity
    gc.collect()

    # Handle missing values
    train.fillna(-999, inplace=True)
    test.fillna(-999, inplace=True)

    # Define categorical features
    categorical_features = [
        'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain',
        'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
        'id_33', 'id_34', 'DeviceType', 'DeviceInfo'
    ]
    categorical_features += [f'id_{i}' for i in range(12, 39)]

    # Encode categorical features
    for col in categorical_features:
        if col in train.columns:
            train[col] = train[col].astype(str)
            test[col] = test[col].astype(str)

            le = LabelEncoder()
            combined_data = pd.concat([train[col], test[col]], axis=0)
            le.fit(combined_data)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

    return train, test

# Load data
train, test = load_data_with_scaling_encoding_dtypes()


In [3]:
X = train.drop(columns=['isFraud', 'TransactionID'])
y = train['isFraud']
X_test = test.drop(columns=['TransactionID'], errors='ignore')

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [4]:
def optuna_cat(trial):
    print(f"[CatBoost][Trial {trial.number}] Starting optimization with params:")
    
    params = {
        'iterations': 10000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'random_strength': trial.suggest_float('random_strength', 0, 1),
        'task_type': 'GPU',
        'devices': '0'
    }
    
    print(f"[CatBoost][Trial {trial.number}] Parameters: {params}")
    
    # Initialize the CatBoost model
    model = CatBoostClassifier(**params, eval_metric='AUC', verbose=100, early_stopping_rounds=100)

    print(f"[CatBoost][Trial {trial.number}] Training started...")
    model.fit(X_train, y_train, eval_set=(X_val, y_val))
    
    print(f"[CatBoost][Trial {trial.number}] Training completed.")
    
    # Predict probabilities and calculate AUC
    val_pred = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, val_pred)
    print(f"[CatBoost][Trial {trial.number}] AUC Score: {auc_score:.4f}")
    
    return auc_score

# Run Optuna study
study_name = "catboost_optimization"
study_cat = optuna.create_study(study_name=study_name, direction="maximize")
study_cat.optimize(optuna_cat, n_trials=50, n_jobs=1)


[I 2025-03-18 16:18:26,719] A new study created in memory with name: catboost_optimization


[CatBoost][Trial 0] Starting optimization with params:
[CatBoost][Trial 0] Parameters: {'iterations': 10000, 'learning_rate': 0.05505036414559644, 'depth': 9, 'l2_leaf_reg': 5.270125117121864, 'bagging_temperature': 0.8568101522402678, 'random_strength': 0.6681621553359959, 'task_type': 'GPU', 'devices': '0'}
[CatBoost][Trial 0] Training started...


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7781611	best: 0.7781611 (0)	total: 365ms	remaining: 1h 47s
100:	test: 0.8949429	best: 0.8949429 (100)	total: 3.64s	remaining: 5m 56s
200:	test: 0.9082919	best: 0.9082919 (200)	total: 6.77s	remaining: 5m 30s
300:	test: 0.9145387	best: 0.9145387 (300)	total: 9.71s	remaining: 5m 12s
400:	test: 0.9199018	best: 0.9199018 (400)	total: 12.7s	remaining: 5m 4s
500:	test: 0.9243885	best: 0.9243885 (500)	total: 15.8s	remaining: 4m 59s
600:	test: 0.9279602	best: 0.9279602 (600)	total: 18.8s	remaining: 4m 54s
700:	test: 0.9313888	best: 0.9313888 (700)	total: 21.8s	remaining: 4m 49s
800:	test: 0.9340028	best: 0.9340028 (800)	total: 24.8s	remaining: 4m 44s
900:	test: 0.9370286	best: 0.9370286 (900)	total: 27.9s	remaining: 4m 41s
1000:	test: 0.9394157	best: 0.9394157 (1000)	total: 30.9s	remaining: 4m 38s
1100:	test: 0.9417224	best: 0.9417224 (1100)	total: 34s	remaining: 4m 35s
1200:	test: 0.9437722	best: 0.9437838 (1199)	total: 37.1s	remaining: 4m 32s
1300:	test: 0.9453894	best: 0.9453894 (

[I 2025-03-18 16:23:01,534] Trial 0 finished with value: 0.9714695244705981 and parameters: {'learning_rate': 0.05505036414559644, 'depth': 9, 'l2_leaf_reg': 5.270125117121864, 'bagging_temperature': 0.8568101522402678, 'random_strength': 0.6681621553359959}. Best is trial 0 with value: 0.9714695244705981.


...

[CatBoost][Trial 48] AUC Score: 0.9455
[CatBoost][Trial 49] Starting optimization with params:
[CatBoost][Trial 49] Parameters: {'iterations': 10000, 'learning_rate': 0.04364957155275623, 'depth': 10, 'l2_leaf_reg': 6.780487860684647, 'bagging_temperature': 0.3853869571046864, 'random_strength': 0.8067089739018423, 'task_type': 'GPU', 'devices': '0'}
[CatBoost][Trial 49] Training started...


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8175166	best: 0.8175166 (0)	total: 96.3ms	remaining: 16m 2s
100:	test: 0.8940826	best: 0.8940826 (100)	total: 5.18s	remaining: 8m 27s
200:	test: 0.9098170	best: 0.9098170 (200)	total: 10.3s	remaining: 8m 22s
300:	test: 0.9172322	best: 0.9172322 (300)	total: 15.1s	remaining: 8m 6s
400:	test: 0.9242669	best: 0.9242669 (400)	total: 20s	remaining: 7m 58s
500:	test: 0.9296616	best: 0.9296616 (500)	total: 25s	remaining: 7m 53s
600:	test: 0.9339913	best: 0.9339916 (599)	total: 29.9s	remaining: 7m 46s
700:	test: 0.9369898	best: 0.9369898 (700)	total: 34.7s	remaining: 7m 39s
800:	test: 0.9406428	best: 0.9406428 (800)	total: 39.6s	remaining: 7m 35s
900:	test: 0.9433327	best: 0.9433330 (899)	total: 44.4s	remaining: 7m 28s
1000:	test: 0.9459379	best: 0.9459379 (1000)	total: 49.4s	remaining: 7m 23s
1100:	test: 0.9483675	best: 0.9483675 (1100)	total: 54.3s	remaining: 7m 18s
1200:	test: 0.9505593	best: 0.9505593 (1200)	total: 59.2s	remaining: 7m 13s
1300:	test: 0.9522498	best: 0.9522498 (1

[I 2025-03-18 19:59:20,497] Trial 49 finished with value: 0.9731257151776262 and parameters: {'learning_rate': 0.04364957155275623, 'depth': 10, 'l2_leaf_reg': 6.780487860684647, 'bagging_temperature': 0.3853869571046864, 'random_strength': 0.8067089739018423}. Best is trial 44 with value: 0.9737512550851547.


[CatBoost][Trial 49] AUC Score: 0.9731


In [5]:
best_cat_params = study_cat.best_params
with open("cat_best_params.json", "w") as f:
    json.dump(best_cat_params, f)

In [6]:
optimized_cat_model = CatBoostClassifier(
    **best_cat_params, 
    eval_metric='AUC', 
    verbose=100, 
    early_stopping_rounds=100
)
optimized_cat_model.fit(
    X_train, y_train, eval_set=(X_val, y_val)
)

0:	test: 0.7847960	best: 0.7847960 (0)	total: 1.62s	remaining: 26m 56s
100:	test: 0.8854692	best: 0.8854692 (100)	total: 2m 48s	remaining: 24m 58s
200:	test: 0.9020941	best: 0.9020941 (200)	total: 5m 33s	remaining: 22m 4s
300:	test: 0.9108555	best: 0.9108555 (300)	total: 8m 20s	remaining: 19m 22s
400:	test: 0.9168915	best: 0.9168915 (400)	total: 11m 7s	remaining: 16m 36s
500:	test: 0.9216929	best: 0.9216929 (500)	total: 13m 55s	remaining: 13m 52s
600:	test: 0.9252240	best: 0.9252240 (600)	total: 16m 43s	remaining: 11m 6s
700:	test: 0.9284482	best: 0.9284482 (700)	total: 19m 30s	remaining: 8m 19s
800:	test: 0.9315267	best: 0.9315267 (800)	total: 22m 19s	remaining: 5m 32s
900:	test: 0.9340169	best: 0.9340185 (899)	total: 25m 5s	remaining: 2m 45s
999:	test: 0.9365404	best: 0.9365404 (999)	total: 27m 50s	remaining: 0us

bestTest = 0.936540362
bestIteration = 999



In [7]:
joblib.dump(optimized_cat_model, "optimized_cat_model.pkl")

['optimized_cat_model.pkl']

In [8]:
val_pred = optimized_cat_model.predict_proba(X_val)[:, 1]
val_predictions = pd.DataFrame({'isFraud': y_val, 'predicted': val_pred})
val_predictions.to_csv("cat_val_predictions.csv", index=False)

In [9]:
val_auc = roc_auc_score(y_val, val_pred)
print(f"Validation AUC: {val_auc:.4f}")

Validation AUC: 0.9365


In [10]:
test_predictions = optimized_cat_model.predict_proba(X_test)[:, 1]

# Generate and save test predictions
submission = pd.DataFrame(
    {'TransactionID': test['TransactionID'], 'isFraud': test_predictions}
)
submission.to_csv("submission.csv", index=False)
submission.to_csv("cat_test_predictions.csv", index=False)

In [11]:
classification_report(y_val,val_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets